# Main 1 : Covid 19

In [1]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
import pyspark.sql.types as T
spark = SparkSession.builder.getOrCreate()

In [2]:
df = spark.read.csv("covid19.csv", header=True, inferSchema=True)
import pandas as pd
import numpy as np

dff = df.toPandas()
#The dataSet is too big .. We will only work with 500 rows !! just to test the streaming 
#print(dff.head(500).to_csv("covid19_10.csv",index=False))
df1 = spark.read.csv("covid19_10.csv", header=True, inferSchema=True)


In [3]:
df.columns

['SNo',
 'ObservationDate',
 'Province',
 'Country',
 'Last_Update',
 'Confirmed',
 'Deaths',
 'Recovered',
 'Active_case']

In [4]:
df.show(2)

+---+---------------+--------+-------+---------------+---------+------+---------+-----------+
|SNo|ObservationDate|Province|Country|    Last_Update|Confirmed|Deaths|Recovered|Active_case|
+---+---------------+--------+-------+---------------+---------+------+---------+-----------+
|  1|     01/22/2020|   Anhui|  China|1/22/2020 17:00|        1|     0|        0|          1|
|  2|     01/22/2020| Beijing|  China|1/22/2020 17:00|       14|     0|        0|         14|
+---+---------------+--------+-------+---------------+---------+------+---------+-----------+
only showing top 2 rows



Step maps a unit of time in the real world. In this case 1 step is 1 hour of time. So we can assume for this example that we have another job that runs every hour and gets all the transactions in that time frame.

In [5]:
df.groupBy("Last_Update").count().show(3)

+-------------------+-----+
|        Last_Update|count|
+-------------------+-----+
|2020-02-10T13:23:06|    1|
|2020-02-10T19:03:02|    3|
|2020-02-21T13:03:09|    1|
+-------------------+-----+
only showing top 3 rows



We can therefore save the output of that job by filtering on each step and saving it to a separate file. 

In [6]:
#%%time
# Countries = df1.select("Active_case").distinct().collect()
# for Active_case in Countries[:]:
 #   _df = df1.where(f"Active_case = {Active_case[0]}")
  #  #by adding coalesce(1) we save the dataframe to one file
   # _df.coalesce(1).write.mode("append").option("header", "true").csv("data")
    

In [7]:
!cd data/ && dir

 Le volume dans le lecteur C n'a pas de nom.
 Le num‚ro de s‚rie du volume est 9E1C-407B

 R‚pertoire de c:\sparkProjectFolder\SUAS-Project-Spark-Streaming\data

29/01/2021  19:32    <DIR>          .
29/01/2021  19:32    <DIR>          ..
29/01/2021  18:37                12 .part-00000-084e2fde-d641-4638-88d5-828fd264d1ff-c000.csv.crc
29/01/2021  19:24                20 .part-00000-09c9943a-1ac7-4ded-b39f-513d3465a2b7-c000.csv.crc
29/01/2021  19:01                12 .part-00000-09f5708d-c851-4ba4-8d1d-aad42fd6a3ad-c000.csv.crc
29/01/2021  18:35                12 .part-00000-09feeb2a-3561-45c3-8f34-e7724685d623-c000.csv.crc
29/01/2021  18:59                12 .part-00000-0b749f2a-6857-4845-afa0-3a871985749e-c000.csv.crc
29/01/2021  19:29                12 .part-00000-0dda2451-38da-47a8-8169-97b15c50e8ad-c000.csv.crc
29/01/2021  19:08                12 .part-00000-0fa1403c-4e3b-4b47-8276-9aa9828880a0-c000.csv.crc
29/01/2021  19:05                12 .part-00000-13748042-45a0-4efc-948a-548

29/01/2021  19:26               144 part-00000-1eb58da7-137c-48ed-b5b5-db95037a6364-c000.csv
29/01/2021  18:37               255 part-00000-20d62593-37c1-4923-98cf-0c02b4d8d794-c000.csv
29/01/2021  19:32               149 part-00000-2292c0d3-588f-4857-97da-60b8ed62795a-c000.csv
29/01/2021  19:29               263 part-00000-22e0359f-03b2-4377-91a0-0e50ac5a28a2-c000.csv
29/01/2021  19:11               147 part-00000-24df11b4-211c-4643-aefd-2ebfeb0a394c-c000.csv
29/01/2021  18:23               928 part-00000-251c9b6b-dc36-4ef2-9a83-a137cd7f28c4-c000.csv
29/01/2021  18:58               204 part-00000-255ba5c6-b778-4bb4-9ffd-f234d8dfd7de-c000.csv
29/01/2021  19:01               206 part-00000-25c1b987-50e8-43fd-82c5-d485d3f3df3f-c000.csv
29/01/2021  18:52               147 part-00000-26bb2e18-ac60-4256-8e00-db41016a6389-c000.csv
29/01/2021  19:28               145 part-00000-28ddc4a0-5658-4a98-8a50-02744f4e5b7e-c000.csv
29/01/2021  19:14               147 part-00000-2a19c72b-9fad-438f-8dee

In [8]:
part = spark.read.csv(
    "data/part-00000-084e2fde-d641-4638-88d5-828fd264d1ff-c000.csv",
    header=True,
    inferSchema=True,
)

In [9]:
part.groupBy("Country").count().show()

+-------+-----+
|Country|count|
+-------+-----+
|  China|    1|
+-------+-----+



Let’s create a streaming version of this input, we'll read each file one by one as if it was a stream.

In [10]:
dataSchema = df1.schema

In [11]:
dataSchema

StructType(List(StructField(SNo,IntegerType,true),StructField(ObservationDate,StringType,true),StructField(Province,StringType,true),StructField(Country,StringType,true),StructField(Last_Update,StringType,true),StructField(Confirmed,IntegerType,true),StructField(Deaths,IntegerType,true),StructField(Recovered,IntegerType,true),StructField(Active_case,DoubleType,true)))

*maxFilesPerTrigger* allows you to control how quickly Spark will read all of the files in the folder. 
In this example we're limiting the flow of the stream to one file per trigger.

In [12]:
streaming = (
    spark.readStream.schema(dataSchema)
    .option("maxFilesPerTrigger", 1)
    .csv("data/")
)

Let's set up a transformation.

The Confirmed column is the recipient ID of the transaction.

In [13]:
confirmed_Country = streaming.groupBy("Confirmed").count().orderBy(F.desc("Confirmed"))

Now that we have our transformation, we need to specify an output sink for the results. For this example, we're going to write to a memory sink which keeps the results in memory.

We also need to define how Spark will output that data. In this example, we'll use the complete output mode (rewriting all of the keys along with their counts after every trigger).

In this example we won't include activityQuery.awaitTermination() because it is required only to prevent the driver process from terminating when the stream is active.

So in order to be able to run this locally in a notebook we won't include it.

In [37]:
activityQuery = (
    confirmed_Country.writeStream.queryName("confirmed_Country")
    .format("memory")
    .outputMode("complete")
    .start()
)



import time

for x in range(10):
    _df = spark.sql(
        "SELECT * FROM confirmed_Country"
    )
    if _df.count() > 0:
        _df.show(10)
    time.sleep(0.5)

+---------+-----+
|Confirmed|count|
+---------+-----+
|      151|    1|
|     null|    1|
+---------+-----+

+---------+-----+
|Confirmed|count|
+---------+-----+
|      173|    1|
|      151|    1|
|     null|    2|
+---------+-----+

+---------+-----+
|Confirmed|count|
+---------+-----+
|      173|    1|
|      151|    1|
|        9|    2|
|        8|   13|
|     null|    3|
+---------+-----+

+---------+-----+
|Confirmed|count|
+---------+-----+
|      173|    1|
|      151|    1|
|       72|    1|
|       70|    2|
|        9|    2|
|        8|   13|
|     null|    4|
+---------+-----+

+---------+-----+
|Confirmed|count|
+---------+-----+
|      173|    1|
|      151|    1|
|       72|    1|
|       70|    2|
|        9|    2|
|        8|   13|
|        1|    1|
|        0|   19|
|     null|    5|
+---------+-----+

+---------+-----+
|Confirmed|count|
+---------+-----+
|      173|    1|
|      151|    1|
|       72|    1|
|       70|    3|
|       69|    2|
|        9|    2|
|    

+---------+-----+
|Confirmed|count|
+---------+-----+
|     3554|    1|
|      549|    1|
|      538|    1|
|      535|    1|
|      237|    1|
|      184|    1|
|      182|    1|
|      173|    1|
|      168|    1|
|      151|    1|
|      139|    1|
|      129|    1|
|      101|    1|
|       99|    1|
|       75|    2|
|       72|    1|
|       70|    3|
|       69|    2|
|       65|    1|
|       53|    1|
|       51|    1|
|       48|    2|
|       46|    1|
|       44|    3|
|       43|    2|
|       36|    1|
|       35|    2|
|       29|    1|
|        9|    2|
|        8|   14|
|        7|   14|
|        1|    1|
|        0|   19|
|     null|   27|
+---------+-----+

+---------+-----+
|Confirmed|count|
+---------+-----+
|     3554|    1|
|      761|    1|
|      549|    1|
|      538|    1|
|      535|    1|
|      237|    1|
|      184|    1|
|      182|    1|
|      173|    1|
|      168|    1|
|      151|    1|
|      139|    1|
|      129|    1|
|      101|    1|
|       9

+---------+-----+
|Confirmed|count|
+---------+-----+
|     7153|    1|
|     3554|    1|
|      761|    1|
|      599|    1|
|      549|    1|
|      538|    1|
|      535|    1|
|      278|    1|
|      277|    2|
|      237|    1|
|      211|    1|
|      184|    1|
|      182|    1|
|      173|    1|
|      168|    1|
|      151|    1|
|      139|    1|
|      129|    1|
|      120|    1|
|      101|    1|
|       99|    1|
|       91|    1|
|       80|    1|
|       75|    2|
|       72|    1|
|       70|    3|
|       69|    2|
|       65|    1|
|       53|    1|
|       51|    1|
|       48|    2|
|       46|    1|
|       44|    3|
|       43|    2|
|       41|    1|
|       40|    1|
|       39|    1|
|       36|    1|
|       35|    2|
|       29|    1|
|       18|    3|
|        9|    2|
|        8|   14|
|        7|   14|
|        2|    1|
|        1|   85|
|        0|   19|
|     null|   40|
+---------+-----+

+---------+-----+
|Confirmed|count|
+---------+-----+
|     715

KeyboardInterrupt: 

In [14]:
Deaths_Country = streaming.groupBy("Deaths").count().orderBy(F.desc("Deaths"))

activityQuery_2 = (
    Deaths_Country.writeStream.queryName("Deaths_Country")
    .format("memory")
    .outputMode("complete")
    .start()
)



import time

for x in range(10):
    _df = spark.sql(
        "SELECT * FROM Deaths_Country"
    )
    if _df.count() > 0:
        _df.show(10)
    time.sleep(0.5)

+------+-----+
|Deaths|count|
+------+-----+
|     0|    1|
|  null|    1|
+------+-----+

+------+-----+
|Deaths|count|
+------+-----+
|     0|    1|
|  null|    1|
+------+-----+

+------+-----+
|Deaths|count|
+------+-----+
|     0|    1|
|  null|    1|
+------+-----+

+------+-----+
|Deaths|count|
+------+-----+
|     0|    1|
|  null|    1|
+------+-----+

+------+-----+
|Deaths|count|
+------+-----+
|     0|    2|
|  null|    2|
+------+-----+

+------+-----+
|Deaths|count|
+------+-----+
|     0|    2|
|  null|    2|
+------+-----+



In [15]:
Recovered_Country = streaming.groupBy("Recovered").count().orderBy(F.desc("Recovered"))

activityQuery_3 = (
    Recovered_Country.writeStream.queryName("Recovered_Country")
    .format("memory")
    .outputMode("complete")
    .start()
)



import time

for x in range(10):
    _df = spark.sql(
        "SELECT * FROM Recovered_Country"
    )
    if _df.count() > 0:
        _df.show(10)
    time.sleep(0.5)

+---------+-----+
|Recovered|count|
+---------+-----+
|        4|    1|
|     null|    1|
+---------+-----+

+---------+-----+
|Recovered|count|
+---------+-----+
|        4|    1|
|     null|    1|
+---------+-----+

+---------+-----+
|Recovered|count|
+---------+-----+
|        4|    1|
|        3|    1|
|     null|    2|
+---------+-----+

+---------+-----+
|Recovered|count|
+---------+-----+
|        4|    1|
|        3|    1|
|        1|    1|
|        0|   14|
|     null|    3|
+---------+-----+

+---------+-----+
|Recovered|count|
+---------+-----+
|        4|    1|
|        3|    1|
|        1|    1|
|        0|   14|
|     null|    3|
+---------+-----+



Check if stream is active

In [16]:
spark.streams.active[0].isActive

True

In [17]:
activityQuery.status

NameError: name 'activityQuery' is not defined

If we  want to turn off the stream we'll run activityQuery.stop() to reset the query for testing purposes.

In [98]:
activityQuery.stop()